In [1]:
!wget -P /data http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz

--2021-01-10 08:06:43--  http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.142.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘/data/speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G   123MB/s    in 20s     

2021-01-10 08:07:04 (115 MB/s) - ‘/data/speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]



In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# For Audio 
import librosa, librosa.display

In [3]:
import tarfile
tar = tarfile.open('/data/speech_commands_v0.02.tar.gz', "r:gz")
tar.extractall('/data/test')
tar.close()

In [4]:
path = '/data/test/'
count = 0
for file in os.listdir(path):
  count+=1

print("Number of words: "+str(count))

path = '/data/test/nine'
count = 0
for file in os.listdir(path):
  count+=1

print("Number of trainind data per word: "+str(count))

Number of words: 41
Number of trainind data per word: 3934


In [5]:
os.listdir(path)[0:5]

['91b03183_nohash_1.wav',
 '40738a2d_nohash_3.wav',
 '541e4079_nohash_0.wav',
 '7c2a3fa4_nohash_0.wav',
 'f822b9bf_nohash_2.wav']

In [6]:
dataLabels = ['tree','house','zero','bed','yes','four','up','stop','no','wow','nine','happy','follow', 'visual','cat','two', 'forward', 'down','right', 'marvin', 'seven', 'go', 'three',  'backward', 'on', 'dog', 'one', 'sheila', 'eight', 'bird', 'six', 'learn', 'off', 'left', 'five']
path = '/data/test/'

In [7]:
len(dataLabels)

35

In [8]:
# 
# Saving Data to numpy array
# 
# 
# convert file to wav2mfcc using the algorithm 
# Mel-frequency cepstral coefficients (MFCC)
def wav2mfcc(file_path, n_mfcc=20, max_len=44):
    wave, sr = librosa.load(file_path, mono=True)
    wave = np.asfortranarray(wave[::3])
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=n_mfcc)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc


def save_data_to_array(path,labels, max_len=44, n_mfcc=20):
    #labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len, n_mfcc=n_mfcc)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)

In [9]:
save_data_to_array(path,['tree']])

Saving vectors of label - 'house':  38%|███▊      | 799/2113 [02:03<03:18,  6.63it/s]

KeyboardInterrupt: ignored

In [ ]:
!cp left.npy '/content/drive/MyDrive/Colab Notebooks/data/'